# Actividad 2 - Clasificación
# Universidad Europea de Madrid
Inteligencia Artificial<br>
Isabel Sutil<br>
M31<br>
Jorge Javier Castilla Coello<br>
Geronimo Basso Sosa<br>
Luis Cabello Casquete<br>
Paula Sáenz de Santa María Diez<br>
Ana Esteban González<br>




# Codigo en común utilizado por todos los apartados.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.base import clone
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import KFold

# Apartado 0
Modificar el dataset para que cada vez que en "prec" se encuente el valor 0,0 se considere que no llueve, para los demas valores se va a considerar que si ha llovido

In [9]:
# Cargamos el fichero necesario para analizar los modelos.
ruta_archivo = 'valores-climatologicos.csv'
fdata = pd.read_table(ruta_archivo, sep=',')

Utilizando una función lambda, para cada valor de ´prec´cambiamos su valor a 0 si la precipitación es 0
cambiamos el valor a 1 si la precipitación es disitnto de 0. Además, creamos un mapeo de indivaticos de provincia para tener valores de tipo int en esa columna envez de strings.

In [3]:
fdata['prec'] = fdata['prec'].apply(lambda x: '0' if x == 0.0 else '1') # 0 si no llovio, 1 si llovio.
states = ['1387E', '1387', '1393', '1351', '1400', '1437O', '1473A', '1428', '1475X', '1505', '1484', '1484C', '1495']
state_map = {state: idx for idx, state in enumerate(sorted(set(states)))}
fdata['indicativo'] = fdata['indicativo'].apply(lambda x: state_map[x])

In [4]:
# Visualización de las primeras filas del dataset.
print(fdata.head())

   indicativo  altitud  tmed prec  tmin  tmax  dir  velmedia  racha  presMax  \
0           2       98  16.9    0  12.0  21.8    1       3.1    8.3   1008.7   
1           2       98  20.5    1  12.5  28.5   22       4.7   11.7   1008.5   
2           2       98  21.4    0  17.6  25.1   22       2.2   10.3   1008.7   
3           2       98  21.4    1  16.0  26.8   99       5.3   10.8   1009.4   
4           2       98  18.6    0  15.7  21.6    4       3.9   11.7   1012.7   

   presMin  
0   1003.9  
1   1005.5  
2   1006.1  
3   1005.4  
4   1009.3  


# Apartado 1
Análisis exploratorio

In [7]:
# Verificamos si hay valores nulos en nuestro dataset.
print(fdata.isna().any())

NameError: name 'fdata' is not defined

In [6]:
# Convertir la columna 'prec' de 'object' a 'categorical'
fdata['prec'] = pd.Categorical(fdata['prec'])
print(fdata.dtypes)

indicativo       int64
altitud          int64
tmed           float64
prec          category
tmin           float64
tmax           float64
dir              int64
velmedia       float64
racha          float64
presMax        float64
presMin        float64
dtype: object


# Apartado 2
Análisis predictivo con Regresión Lineal

In [7]:
# Crear un data set especifico con valores de tipo float para la regression lineal.
floats_dataframe = fdata.astype(float)

# Convertir la columna 'prec' de 'float' a 'categorical'
floats_dataframe['prec'] = pd.Categorical(floats_dataframe['prec'])
print(floats_dataframe.dtypes)

indicativo     float64
altitud        float64
tmed           float64
prec          category
tmin           float64
tmax           float64
dir            float64
velmedia       float64
racha          float64
presMax        float64
presMin        float64
dtype: object


In [6]:
X = floats_dataframe.drop('prec', axis=1)  # Features
y = floats_dataframe['prec']  # Variable de salida

# División en 80/20 para entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_val))

# Crear conjuntos de entrenamiento y prueba
conjunto_train = pd.concat([X_train, y_train], axis=1)
conjunto_val = pd.concat([X_val, y_val], axis=1)

# Creamos dos conjuntos para poner las predicciones del train y test
conjunto_train_eval = conjunto_train.copy()
conjunto_validation_eval = conjunto_val.copy()

# Método de control (k-fold cross-validation)
metodo_control_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

NameError: name 'floats_dataframe' is not defined

In [9]:
# Definir el modelo de Regresion Lineal.
liner_regression_model = LinearRegression()

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalar las características
    ('model', liner_regression_model)        # Modelo de regresión logística
])

# Obtener todos los modelos de regresión lineal entrenados durante la validación cruzada
liner_regression_models = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Imprimir información sobre el modelo
for i, modelo in enumerate(liner_regression_models):
    print(f"\nLinear Regression Model trained in fold {i + 1}:")
    precision = modelo.score(X_train, y_train)
    print("Precisión en el step:", precision)


Linear Regression Model trained in fold 1:
Precisión en el step: 0.2897079771771981

Linear Regression Model trained in fold 2:
Precisión en el step: 0.28972270380803866

Linear Regression Model trained in fold 3:
Precisión en el step: 0.2897645807853185

Linear Regression Model trained in fold 4:
Precisión en el step: 0.28969591901494307

Linear Regression Model trained in fold 5:
Precisión en el step: 0.2897227144081307

Linear Regression Model trained in fold 6:
Precisión en el step: 0.2897278011997909

Linear Regression Model trained in fold 7:
Precisión en el step: 0.2896186774824374

Linear Regression Model trained in fold 8:
Precisión en el step: 0.2897401429317781

Linear Regression Model trained in fold 9:
Precisión en el step: 0.28967344783597193

Linear Regression Model trained in fold 10:
Precisión en el step: 0.28973398250702564


# Apartado 3
Análisis predictivo con KNN

Utilizamos el dataset del Apartado 2, dado que el mismo ya tiene sus valores convertidos como floats y necesitamos únicamente valores de tipo float o int en nuestro dataset para útilizar K-NN.

In [10]:
# Definir el modelo K-NN.
k_nn_model = KNeighborsRegressor(n_neighbors=10)

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalar las características
    ('model', k_nn_model)        # Modelo de regresión logística
])

# Obtener todos los modelos de regresión lineal entrenados durante la validación cruzada
k_nn_models = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Imprimir información sobre el modelo
for i, modelo in enumerate(k_nn_models):
    print(f"\nK-NN Model trained in fold {i + 1}:")
    precision = modelo.score(X_train, y_train)
    print("Precisión en el step:", precision)


K-NN Model trained in fold 1:
Precisión en el step: 0.47782784069376016

K-NN Model trained in fold 2:
Precisión en el step: 0.4775850667089606

K-NN Model trained in fold 3:
Precisión en el step: 0.47858650939625913

K-NN Model trained in fold 4:
Precisión en el step: 0.48091020325077016

K-NN Model trained in fold 5:
Precisión en el step: 0.4795055823387151

K-NN Model trained in fold 6:
Precisión en el step: 0.4784737929033164

K-NN Model trained in fold 7:
Precisión en el step: 0.48040297903252804

K-NN Model trained in fold 8:
Precisión en el step: 0.4797136743256861

K-NN Model trained in fold 9:
Precisión en el step: 0.47812697292503126

K-NN Model trained in fold 10:
Precisión en el step: 0.4775330437122177


# Apartado 4
Análisis predictivo con un árbol de decisión

In [5]:
# Definir el modelo de Árbol de Decisión.
decision_tree_model = DecisionTreeRegressor()

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),      # Escalar las características
    ('model', decision_tree_model)    # Modelo de Árbol de Decisión
])

# Obtener todos los modelos de Árbol de Decisión entrenados durante la validación cruzada
decision_tree_models = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Imprimir información sobre el modelo
for i, modelo in enumerate(decision_tree_models):
    print(f"\nDecision Tree Model trained in fold {i + 1}:")
    precision = modelo.score(X_train, y_train)
    print("Precisión en el step:", precision)

NameError: name 'metodo_control_cv' is not defined

# Apartado 5
Análisis predictivo con SVM

In [ ]:
# Definir el modelo SVM para regresión.
svm_model = SVR()

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalar las características
    ('model', svm_model)            # Modelo SVM para regresión
])

# Obtener todos los modelos SVM entrenados durante la validación cruzada
svm_models = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Imprimir información sobre el modelo
for i, modelo in enumerate(svm_models):
    print(f"\nSVM Model trained in fold {i + 1}:")
    precision = modelo.score(X_train, y_train)
    print("Precisión en el step:", precision)

# Apartado 6
Análisis predictivo con Naive Bayes.

In [11]:
# Definir el modelo de Naive Bayes.
naive_bayes_model = GaussianNB()

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalar las características
    ('model', naive_bayes_model)        # Modelo de regresión logística
])

# Obtener todos los modelos de Naive Bayes entrenados durante la validación cruzada
naive_bayes_models = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Imprimir información sobre el modelo GaussianNB.
for i, modelo in enumerate(naive_bayes_models):
    print(f"\nModelo de Naive Bayes entrenado en el pliegue {i + 1}:")
    # Printing accuracy or other performance metrics
    accuracy = modelo.score(X_train, y_train)
    print("Precisión:", accuracy)


Modelo de Naive Bayes entrenado en el pliegue 1:
Precisión: 0.6689423493714781

Modelo de Naive Bayes entrenado en el pliegue 2:
Precisión: 0.6728435197225835

Modelo de Naive Bayes entrenado en el pliegue 3:
Precisión: 0.6673168617251842

Modelo de Naive Bayes entrenado en el pliegue 4:
Precisión: 0.6686172518422193

Modelo de Naive Bayes entrenado en el pliegue 5:
Precisión: 0.6712180320762896

Modelo de Naive Bayes entrenado en el pliegue 6:
Precisión: 0.6654746423927178

Modelo de Naive Bayes entrenado en el pliegue 7:
Precisión: 0.6702427394885132

Modelo de Naive Bayes entrenado en el pliegue 8:
Precisión: 0.6741439098396186

Modelo de Naive Bayes entrenado en el pliegue 9:
Precisión: 0.6736020806241872

Modelo de Naive Bayes entrenado en el pliegue 10:
Precisión: 0.6675335934113568


# Apartado 7
Análisis comparativo de todos los modelos en train y en validation